# Data Quality Checks

**Purpose:** Verify data correctness before analysis

**Tasks:**
- Check for missing values
- Find impossible values (age = -10, heart rate = 900)
- Identify duplicates

In [1]:
import pandas as pd
import numpy as np
import sqlite3
# import matplotlib.pyplot as plt
# import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
# sns.set_style('whitegrid')

## Load Database

**Important:** Load data BEFORE the train/test split in Transformation.py

In [2]:
# Connect to database and load full dataset
conn = sqlite3.connect('../databases/nhanes_1st.db')
df = pd.read_sql_query('SELECT * FROM raw_dataset', conn)
conn.close()
print(f"Dataset shape: {df.shape}")

Dataset shape: (34097, 29)


## 1. Missing Values Analysis

In [3]:
# Analyze missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing': missing, 'Percent': missing_pct})
missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Percent', ascending=False)

print('Missing Values Summary:\n')
print('' + missing_df.to_string())

Missing Values Summary:

                            Missing    Percent
high_glucose_mg_dl            20281  59.480306
high_triglycerides_mg_dl      19884  58.315981
smoking_status                19761  57.955245
alcohol_drinks_per_week       10509  30.820893
heart_rate_bpm                 6622  19.421063
liver_ast_U_L                  5445  15.969147
potassium_mmol_L               5424  15.907558
bilirubin_mg_dl                5404  15.848902
high_blood_pressure            5399  15.834238
liver_dysfunction              5389  15.804910
uric_acid_mg_dl                5384  15.790245
liver_ggt_U_L                  5382  15.784380
creatinine_mg_dl               5380  15.778514
sodium_mmol_L                  5341  15.664135
cholesterol_mg_dl              5242  15.373787
low_hdl_mg_dl                  5242  15.373787
high_waist_circumference       4957  14.537936
income_ratio                   4606  13.508520
platelets_count                4605  13.505587
white_blood_cells_count        4605

In [4]:
# Highlight high missing rates
high_missing = missing_df[missing_df['Percent'] > 50]
print(f'\nColumns with >50% missing: {len(high_missing)}')
if len(high_missing) > 0:
    print('' + high_missing.to_string())


Columns with >50% missing: 3
                          Missing    Percent
high_glucose_mg_dl          20281  59.480306
high_triglycerides_mg_dl    19884  58.315981
smoking_status              19761  57.955245


## 2. Impossible Values Detection

In [5]:
# Define valid ranges for clinical measurements
rules = {
    'age': (0, 115),
    'body_mass_index': (10, 100),
    'height_cm': (50, 250),
    'heart_rate_bpm': (30, 200),
    'white_blood_cells_count': (0, 50),
    'platelets_count': (20, 1000),
    'hemoglobin_g_dl': (5, 20),
    'creatinine_mg_dl': (0.3, 15),
    'liver_ast_U_L': (5, 500),
    'bilirubin_mg_dl': (0.1, 20),
    'liver_ggt_U_L': (5, 500),
    'uric_acid_mg_dl': (1, 15),
    'sodium_mmol_L': (120, 160),
    'potassium_mmol_L': (2.5, 6.0),
    'cholesterol_mg_dl': (50, 500),
}

# Check for impossible values
results = []
for col, (min_val, max_val) in rules.items():
    total = df[col].notna().sum()
    impossible = df[(df[col] < min_val) | (df[col] > max_val)][col]
    if len(impossible) > 0:
        results.append({
            'Column': col,
            'Impossible': len(impossible),
            'Total': total,
            # 'Range': f'[{min_val}, {max_val}]'
        })

if results:
    impossible_df = pd.DataFrame(results)
    print('Impossible values found:\n')
    print(impossible_df.to_string(index=False))
else:
    print('No impossible values found!')

Impossible values found:

           Column  Impossible  Total
 creatinine_mg_dl           6  28717
    liver_ast_U_L           2  28652
  bilirubin_mg_dl          29  28693
    liver_ggt_U_L          66  28715
  uric_acid_mg_dl           6  28713
    sodium_mmol_L           3  28756
 potassium_mmol_L           6  28673
cholesterol_mg_dl           5  28855


## 3. Duplicate Records

In [6]:
# Check for duplicate rows
duplicates = df[df.duplicated(keep=False)]
print(f'Total duplicate rows: {len(duplicates)} out of {len(df)}')

Total duplicate rows: 11679 out of 34097


## Summary

In [7]:
print('='*60)
print('SUMMARY')
print('='*60)
print(f'Total records: {len(df):,}')
print(f'Columns with missing values: {len(missing_df)}')
print(f'Columns with impossible values: {len(results) if results else 0}')
print(f'Duplicate rows: {len(duplicates)}')

SUMMARY
Total records: 34,097
Columns with missing values: 25
Columns with impossible values: 8
Duplicate rows: 11679
